# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to edward30@163.com and will expire on September 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Edward\AppData\Local\Temp\graphlab_server_1478508783.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.538088     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.587090     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [20]:
model_all.coefficients[9:18]

name,index,value,stderr
floors_square,None,0.0,None
waterfront,None,0.0,None
view,None,0.0,None
condition,None,0.0,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None
sqft_basement,None,0.0,None
yr_built,None,0.0,None
yr_renovated,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [6]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [7]:
import numpy as np

In [8]:
for i in np.logspace(1, 7, num=13):
    model_l1=graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=i, verbose = False)
    predictions = model_l1.predict(validation)
    test_rss = sum((predictions-validation['price'])**2)
    print i
    print test_rss
    print (model_l1.coefficients['value']).nnz()

10.0
6.25766285142e+14
18
31.6227766017
6.25766285362e+14
18
100.0
6.25766286058e+14
18
316.227766017
6.25766288257e+14
18
1000.0
6.25766295212e+14
18
3162.27766017
6.25766317206e+14
18
10000.0
6.25766386761e+14
18
31622.7766017
6.25766606749e+14
18
100000.0
6.25767302792e+14
18
316227.766017
6.25769507644e+14
18
1000000.0
6.25776517727e+14
18
3162277.66017
6.25799062845e+14
18
10000000.0
6.25883719085e+14
18


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [27]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [9]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [10]:
i = 0
li = []
for l1_penalty in l1_penalty_values:

    model_2 = graphlab.linear_regression.create(training, features=all_features, l1_penalty=l1_penalty, l2_penalty=0,
                                                target='price',
                                     validation_set = None, verbose = False)
    li.insert(i,model_2['coefficients']['value'].nnz())
    print 'l1_penalty: %d, non-zero parameters:%d' % (l1_penalty,model_2['coefficients']['value'].nnz())

l1_penalty: 100000000, non-zero parameters:18
l1_penalty: 127427498, non-zero parameters:18
l1_penalty: 162377673, non-zero parameters:18
l1_penalty: 206913808, non-zero parameters:18
l1_penalty: 263665089, non-zero parameters:17
l1_penalty: 335981828, non-zero parameters:17
l1_penalty: 428133239, non-zero parameters:17
l1_penalty: 545559478, non-zero parameters:17
l1_penalty: 695192796, non-zero parameters:17
l1_penalty: 885866790, non-zero parameters:16
l1_penalty: 1128837891, non-zero parameters:15
l1_penalty: 1438449888, non-zero parameters:15
l1_penalty: 1832980710, non-zero parameters:13
l1_penalty: 2335721469, non-zero parameters:12
l1_penalty: 2976351441, non-zero parameters:10
l1_penalty: 3792690190, non-zero parameters:6
l1_penalty: 4832930238, non-zero parameters:5
l1_penalty: 6158482110, non-zero parameters:3
l1_penalty: 7847599703, non-zero parameters:1
l1_penalty: 10000000000, non-zero parameters:1


In [15]:
l1_penalty_values = np.logspace(9, 10, num=50)

In [16]:
i = 0
li = []
for l1_penalty in l1_penalty_values:

    model_2 = graphlab.linear_regression.create(training, features=all_features, l1_penalty=l1_penalty, l2_penalty=0,
                                                target='price',
                                     validation_set = None, verbose = False)
    li.insert(i,model_2['coefficients']['value'].nnz())
    print 'l1_penalty: %d, non-zero parameters:%d' % (l1_penalty,model_2['coefficients']['value'].nnz())

l1_penalty: 1000000000, non-zero parameters:16
l1_penalty: 1048113134, non-zero parameters:15
l1_penalty: 1098541141, non-zero parameters:15
l1_penalty: 1151395399, non-zero parameters:15
l1_penalty: 1206792640, non-zero parameters:15
l1_penalty: 1264855216, non-zero parameters:15
l1_penalty: 1325711365, non-zero parameters:15
l1_penalty: 1389495494, non-zero parameters:15
l1_penalty: 1456348477, non-zero parameters:15
l1_penalty: 1526417967, non-zero parameters:15
l1_penalty: 1599858719, non-zero parameters:15
l1_penalty: 1676832936, non-zero parameters:15
l1_penalty: 1757510624, non-zero parameters:13
l1_penalty: 1842069969, non-zero parameters:13
l1_penalty: 1930697728, non-zero parameters:13
l1_penalty: 2023589647, non-zero parameters:13
l1_penalty: 2120950887, non-zero parameters:13
l1_penalty: 2222996482, non-zero parameters:12
l1_penalty: 2329951810, non-zero parameters:12
l1_penalty: 2442053094, non-zero parameters:11
l1_penalty: 2559547922, non-zero parameters:11
l1_penalty: 2

In [58]:
model_2 = graphlab.linear_regression.create(training, features=all_features, l1_penalty=3414000000, l2_penalty=0,
                                                target='price',
                                     validation_set = None, verbose = False)
print 'l1_penalty: %d, non-zero parameters:%d' % (l1_penalty,model_2['coefficients']['value'].nnz())

l1_penalty: 10000000000, non-zero parameters:7


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [60]:
#l1_penalty: 3393221771, non-zero parameters:8
#l1_penalty: 3556480306, non-zero parameters:7
#l1_penalty: 3727593720, non-zero parameters:6

l1_penalty = 3393000000
while l1_penalty < 3727593720:
    model_2 = graphlab.linear_regression.create(training, features=all_features, l1_penalty=l1_penalty, l2_penalty=0,
                                                target='price',
                                     validation_set = None, verbose = False)
    if model_2['coefficients']['value'].nnz() == 7:
        print 'l1_penalty: %d, non-zero parameters:%d' % (l1_penalty,model_2['coefficients']['value'].nnz())
        break
    l1_penalty+=1000000

l1_penalty: 3414000000, non-zero parameters:7


In [61]:
#l1_penalty: 3393221771, non-zero parameters:8
#l1_penalty: 3556480306, non-zero parameters:7
#l1_penalty: 3727593720, non-zero parameters:6

l1_penalty = 3727593720
while l1_penalty > 3393221771:
    model_2 = graphlab.linear_regression.create(training, features=all_features, l1_penalty=l1_penalty, l2_penalty=0,
                                                target='price',
                                     validation_set = None, verbose = False)
    if model_2['coefficients']['value'].nnz() == 7:
        print 'l1_penalty: %d, non-zero parameters:%d' % (l1_penalty,model_2['coefficients']['value'].nnz())
        break
    l1_penalty-=1000000

l1_penalty: 3613593720, non-zero parameters:7


In [62]:
l1_penalty_min = 3414000000
l1_penalty_max = 3613000000

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [63]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [66]:
for l1_penalty in l1_penalty_values:
    model_3 = graphlab.linear_regression.create(training, features=all_features, l1_penalty=l1_penalty, l2_penalty=0,
                                                target='price', validation_set = None, verbose = False)
    predictions = model_3.predict(validation)
    rss = sum((predictions-validation['price'])*(predictions-validation['price']))
    print 'l1_penalty = %d, rss = %d' % (l1_penalty,rss)
    print model_2['coefficients']['value'].nnz()

l1_penalty = 3414000000, rss = 1044112889884659
7
l1_penalty = 3424473684, rss = 1044952352831229
7
l1_penalty = 3434947368, rss = 1045792809125775
7
l1_penalty = 3445421052, rss = 1046634238261443
7
l1_penalty = 3455894736, rss = 1047535125996034
7
l1_penalty = 3466368421, rss = 1048439565654775
7
l1_penalty = 3476842105, rss = 1049453815653044
7
l1_penalty = 3487315789, rss = 1050628884134669
7
l1_penalty = 3497789473, rss = 1051805969325682
7
l1_penalty = 3508263157, rss = 1052985071226084
7
l1_penalty = 3518736842, rss = 1054166189835875
7
l1_penalty = 3529210526, rss = 1055349325155063
7
l1_penalty = 3539684210, rss = 1056533380759380
7
l1_penalty = 3550157894, rss = 1057701129492810
7
l1_penalty = 3560631578, rss = 1058870813826982
7
l1_penalty = 3571105263, rss = 1060042433761897
7
l1_penalty = 3581578947, rss = 1061215989297547
7
l1_penalty = 3592052631, rss = 1062435007831116
7
l1_penalty = 3602526315, rss = 1063706720718790
7
l1_penalty = 3613000000, rss = 1064980782285497
7


In [67]:
model_2['coefficients']

name,index,value,stderr
(intercept),None,230955.489515,None
bedrooms,None,2.82635822964,None
bedrooms_square,None,0.0,None
bathrooms,None,14936.0924776,None
sqft_living,None,31.5677979764,None
sqft_living_sqrt,None,644.949566952,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?